<a href="https://colab.research.google.com/github/WilliamAshbee/DataminingAssignments/blob/main/DataMiningFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/


In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import sklearn as sk
from sklearn import preprocessing
from torch.utils.data import Dataset
import torch

class CustomMedDataset(Dataset):
    def __init__(self):
        url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
        df_clev = pd.read_csv(url,header=None,error_bad_lines=False)
        questionmark = {'?':'-10000'}

        dfcn = df_clev
        dfcn = dfcn.replace({11:questionmark,12:questionmark})
        dfcn = dfcn.to_numpy().astype(float)
        y = dfcn[:,13]

        X = dfcn[:,:13]

        scaler = preprocessing.StandardScaler().fit(X)
        X = scaler.fit_transform(X)

        self.X = torch.from_numpy(X).cuda().float()
        self.y = torch.from_numpy(y).cuda().long() 
        print(self.X.shape)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx,:],self.y[idx]


In [3]:
a = CustomMedDataset()

torch.Size([303, 13])


In [4]:
for i in range(len(a)):
  print(a[i][1])

tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
tensor(4, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
t

In [5]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([2, 1, 1])

In [6]:
#comment from william ashbee: 
#a large portion of the code below may have been taken from the solution
#https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/
#essentially, I wanted to use sklearn's cross validation and write my model in pytorch rather than use
#sklearn's mlp. This code allowed me to do this, which I didn't know was possible. 

import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold
#import torch.optim.lr_scheduler.ReduceLROnPlateau as rop

#scheduler = rop(optimizer, 'min')

def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.inp_len = 13
    self.layers = nn.Sequential(
      nn.Linear(self.inp_len, self.inp_len),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(self.inp_len, self.inp_len),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(self.inp_len, 5),
      nn.Softmax() #https://www.cs.toronto.edu/~lczhang/321/tut/tut04.pdf
      )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
  
  
if __name__ == '__main__':
  
  # Configuration options
  k_folds = 5
  num_epochs = 200
  loss_function = nn.CrossEntropyLoss()#https://www.cs.toronto.edu/~lczhang/321/tut/tut04.pdf
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  dataset = CustomMedDataset()
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)
    
  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network
    network = MLP().cuda()
    network.apply(reset_weights)
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
    
    # Run the training loop for defined number of epochs
    for epoch in range(1, num_epochs):
      if epoch%50==0:
        for g in optimizer.param_groups:
          g['lr'] = g['lr']/10.0
 
      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = network(inputs)
        
        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
      if epoch % 10 == 9:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss/10 ))
          current_loss = 0.0
          
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)
    
  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')


torch.Size([303, 13])
--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=13, out_features=13, bias=True)
Reset trainable parameters of layer = Linear(in_features=13, out_features=13, bias=True)
Reset trainable parameters of layer = Linear(in_features=13, out_features=5, bias=True)
Starting epoch 2
Starting epoch 3
Starting epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Loss after mini-batch    25: 3.601
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Loss after mini-batch    25: 3.352
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Loss after mini-batch    25: 3.276
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Loss after mini-batch    25: 3.262
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50
Loss after mini-batch    25: 3.243
St